In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
data = pd.read_csv('../raw_data/state_sightings.csv', low_memory=False)

In [3]:
def value_count_func (series_):
    return len(series_.value_counts())

def most_freq_func (series_):
    x_ = series_.value_counts()
    return list(x_.keys())[0]

def state_season_df (data_, state_):
    
    data_state = data_[data_['state'] == state_]
    
    data_state_season = data_state.groupby(by='year_season').agg({'datetime':'count','duration (seconds)':'mean',
                                          'city':value_count_func,'shape':most_freq_func,'day':value_count_func,
                                          'state_population':'first', 'state_military_bases':'first','state_cities':'first'})
    
    data_state_season.rename(columns={'datetime':'sightings_period','duration (seconds)':'avg_duration(seconds)',
                                  'city':'sightings_cities','day':'sightings_days_period'},inplace=True)
    
    data_state_season['state'] = state_
    
    return data_state_season.reset_index().set_index('state')

def previous_sightings (x_, idx_):
    
    row_index_state, row_index_year, row_index_season = idx_.split('-') 
    
    period_1 = f"{row_index_state}-{int(row_index_year)-1}-{row_index_season}"
    period_2 = f"{row_index_state}-{int(row_index_year)-2}-{row_index_season}"
    period_3 = f"{row_index_state}-{int(row_index_year)-3}-{row_index_season}"

    try:
        res_1 = x_.loc[period_1].sightings_days_period
    except:
        res_1 = 0
        
    try:
        res_2 = x_.loc[period_2].sightings_days_period
    except:
        res_2 = 0
        
    try:
        res_3 = x_.loc[period_3].sightings_days_period
    except:
        res_3 = 0
    
    return res_1, res_2, res_3    

In [4]:
STATES_ = list(data.state.unique()) 

x_ = pd.DataFrame()

for st_ in STATES_:
    
    y_ = state_season_df (data, st_)
    x_ = pd.concat([x_,y_])
    
x_ = x_.reset_index()

x_['state_season'] = x_['state']+ '-' + x_['year_season']

# x_.drop(columns=['year_season','state'], inplace=True)

x_.set_index('state_season',inplace=True)

x_['sigthing_days_1_year_before'] = None
x_['sigthing_days_2_year_before'] = None
x_['sigthing_days_3_year_before'] = None

In [5]:
indexes_ = list(x_.index)

for i in indexes_:
    
    res1, res2, res3 = previous_sightings (x_, i)
    
    x_.loc[i,'sigthing_days_1_year_before'] = res1
    x_.loc[i,'sigthing_days_2_year_before'] = res2
    x_.loc[i,'sigthing_days_3_year_before'] = res3


In [6]:
x_[x_['state'] == 'tx']

,state,year_season,sightings_period,avg_duration(seconds),sightings_cities,shape,sightings_days_period,state_population,state_military_bases,state_cities,sigthing_days_1_year_before,sigthing_days_2_year_before,sigthing_days_3_year_before
state_season,,,,,,,,,,,,,
tx-1910-summer,tx,1910-summer,1,120.000000,1,cigar,1,33717239.0,5.0,1679.0,0,0,0
tx-1945-summer,tx,1945-summer,1,7200.000000,1,sphere,1,33717239.0,5.0,1679.0,0,0,0
tx-1946-spring,tx,1946-spring,1,20.000000,1,disk,1,33717239.0,5.0,1679.0,0,0,0
tx-1947-winter,tx,1947-winter,1,4.000000,1,other,1,33717239.0,5.0,1679.0,0,0,0
tx-1948-summer,tx,1948-summer,1,10.000000,1,disk,1,33717239.0,5.0,1679.0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
tx-2020-spring,tx,2020-spring,62,960.967742,44,Light,39,33717239.0,5.0,1679.0,30,13,28
tx-2020-summer,tx,2020-summer,41,585.365854,31,Light,34,33717239.0,5.0,1679.0,24,21,15
tx-2020-winter,tx,2020-winter,59,611.186441,48,Light,32,33717239.0,5.0,1679.0,25,13,20


In [7]:
x_.to_csv('../raw_data/features_by_state_season.csv',header=True,index=False)